<center>
    <header>
        <h1>Audio Classifier Python</h1>
        <h2>Marco Bondaschi, Mauro Conte</h2>
        <h4>Laboratorio di Telecomunicazioni</h4>
        <h5>Università Degli Studi Di Brescia</h5>
        <h6>13/03/2017</h6>
    </header>
</center>
<hr><hr>

<h2>Descrizione</h2>

Per questa prima esperienza di laboratorio, abbiamo realizzato un software di classificazione audio, realizzato in Python, che permettesse di analizzare e distinguere un file musicale da uno di parlato.

Per realizzare questo compito abbiamo usufruito di un database di 40 file audio .wav, MONO e con frequenza di campionamento pari a 16000 Hz, già suddivisi in 20 file di musica (dalla durata di 10 s l'uno) e 20 file di parlato (dalla durata di 3 s l'uno).

Abbiamo inoltre in dotazione un database di 10 suoni incogniti, sempre MONO a 16000Hz, che il nostro software dovrà essere in grado di dividere in musica o parlato. A questi 10 file abbiamo ne abbiamo aggiunti altri due, uno di musica e uno di parlato, per verificare con più precisione l'accuratezza del programma.

Il primo passo per creare il software di classificazione è la fase di training, attraverso la quale si addestra il software a distinguere musica e parlato facendogli analizzare i 40 file già suddivisi nelle due categorie.

In secondo luogo si attua la fase di test, in cui si consegnano al software i 10 file incogniti, che verranno analizzati dal programma e suddivisi in musica o parlato.


<h3>Strumenti</h3>

Per raggiungere questi obiettivi, è stato necessario utilizzare le estensioni Scipy e Numpy, che implementano funzioni matematiche di alto livello. In particolare abbiamo utilizzato due funzioni fondamentali: spectrogram e svd. 

In [61]:
import scipy.signal
import scipy.linalg
import scipy.io.wavfile
import numpy as np
import os
import sys

<h3>Spectrogram</h3>
La prima serve per ottenere lo spettrogramma di un segnale audio, che consiste nel suddividere il segnale in intervalli di tempo uguali, finestrarli utilizzando una delle numerose finestre disponibili (nel nostro caso la finestra di Hamming), quindi calcolare la FFT di questi segmenti, utilizzando un numero di campioni a scelta. Da questo procedimento si ottiene perciò una matrice, dove ogni colonna contiene la FFT del segmento di segnale designato (e dunque il numero di righe della matrice equivale al numero di campioni della FFT, mentre il numero di colonne equivale al numero di segmenti in cui è stato suddiviso il segnale).

<h3>SVD</h3>
La seconda funzione, invece, esegue la Decomposizione ai Valori Singolari (SVD, Singular Value Decomposition) su una matrice $M\in{\mathbb{C}^{(m,n)}}$

Questa pratica consiste in una fattorizzazione della matrice $M$ in un prodotto tra tre matrici $M=USV^{*}$, dove:
<ul>
    <li>
$U\in{\mathbb{C}^{(m,n)}}$ è una matrice unitaria (ovvero una matrice il cui prodotto con la sua matrice trasposta coniugata dà la matrice unitaria)
    </li>
    <li>
$S$ è una matrice diagonale  $\in\mathbb{C}^{(m,n)}$
    </li>
    <li>
$V^{*}$ è la trasposta coniugata di una matrice unitaria $V\in{\mathbb{C}^{(n,n)}}$
    </li>
</ul>

Gli elementi sulla diagonale principale di $S$ sono le radici quadrate degli autovalori associati agli autovettori che si trovano sulle colonne di $V^*$.

<h3>Funzioni implementate</h3>

Ritorna una lista di tuple contenenti SampleRate, Data, FileName data una path.

In [62]:
def load_audio(path):
    audio_list = []
    for filename in os.listdir(path):
        rate, data = scipy.io.wavfile.read(path + filename) 
        audio_list.append((rate, data, filename))
    return audio_list

Ritorna lo spettrogramma  di un segnale dati il SampleRate e Data.

Come impostazioni per lo spettrogramma abbiamo seguito la documentazione dello standard mpeg7 allegata, paragrafo 1.1.2.3.4. Poichè la frequenza di campionamento dei file è pari a 16000 Hz, abbiamo ottenuto una hopsize (corrispondente al valore noverlap della funzione di Numpy) pari a 160, la lunghezza della finestra pari alla più piccola potenza di 2 maggiore di 3 hopsize, ovvero 512 campioni, che è anche il numero di campioni della FFT. La finestra utilizzata, come da specifiche, è quella di Hamming.
Inolte, abbiamo calcolato lo spettrogramma in scala logaritmica in modo da velocizzare le operazioni computazionali.

In [63]:
def get_spectrogram(data, rate):
    f, t, sxx = scipy.signal.spectrogram(data, rate, 'hamming', 512, 160, 512)
    spectrogram = 10 * np.log10(sxx + sys.float_info.min)
    return spectrogram

Ritorna lo spettrogramma di un insieme di audio data la lista.

In [64]:
def get_spectrogram_from_list(collection):
    spectrogram = None
    for rate,data,f_name in collection:
        if spectrogram is None: spectrogram = get_spectrogram(data,rate)
        else: scipy.hstack((spectrogram, get_spectrogram(data,rate)))
    return spectrogram

Ritorna una matrice che ha per colonne i vettori di una base ridotta dello spazio vettoriale che genera lo spettrogramma dato lo stesso e la percentuale di energia.

In [65]:
def get_base_from_spectrogram(spectrogram,percent):
    u, s, v = np.linalg.svd(spectrogram)
    energy_e_value = np.sum(s)
    current_sum = 0
    percent_sum = energy_e_value * percent / 100
    for i in range(len(s)):
        current_sum += s[i]
        if current_sum >= percent_sum:
            break
    base = u[:, range(0, i)]
    return base

Ritorna la feature relativa ad un audio ricavata dal suo spettrogramma e dalla base.

In [66]:
def get_feature_mean(spectrogram, base):
    mean_vector = np.zeros(base.shape[1])
    coefficient_on_base = np.dot(spectrogram.T, base)
    mean_vector += np.mean(coefficient_on_base, 0)
    return mean_vector

Ritorna la feature relativa ad una lista di audio ricavata dal suo spettrogramma e dalla base.

In [67]:
def get_feature_mean_from_list(collection, base):
    mean_vector = np.zeros(base.shape[1])
    for rate, data, filename in collection:
        spectrogram = get_spectrogram(data,rate)
        mean_vector += get_feature_mean(spectrogram,base)
    return mean_vector / len(collection)

Calcola l'approssimazione dello spettrogramma su una base, quindi ritorna l'errore tra lo spettrogramma originale e l'approssimato.

In [68]:
def get_error_base(spectrogram, base):
    coefficient_on_base = np.dot(spectrogram.T, base)
    spectrogram_ric = np.dot(coefficient_on_base, base.T).T
    error = np.linalg.norm(spectrogram - spectrogram_ric)
    return error

Per implementare il training abbiamo utilizzato tre diversi metodi.

<h3>Primo Metodo</h3>

Innanzitutto abbiamo caricato i file audio di musica e di parlato:

In [69]:
path_db = os.getcwd()+'/05_AudioClassifier_Pdf/05_AudioClassifier_Pdf/database/'
path_music = path_db + 'music/'
path_speech = path_db + 'speech/'
path_unknowns = path_db.replace('database/', '') + 'unknownSounds/'

music = load_audio(path_music)
speech = load_audio(path_speech)

Quindi abbiamo calcolato lo spettrogramma per ogni file, utilizzando le impostazioni descritte in precedenza.
A questo punto abbiamo concatenato tra loro tutti gli spettrogrammi dei file di musica, e poi abbiamo ripetuto la stessa cosa per i file di parlato, in modo da ottenere due classi distinte che rappresentassero le due categorie. Algebricamente parlando, le due classi risultano essere due sottospazi.

In [70]:
spectrogram_music = get_spectrogram_from_list(music)
spectrogram_speech = get_spectrogram_from_list(speech)

Il passo successivo è stato quello di ottenere due basi di vettori che generassero i sottospazi; abbiamo perciò calcolato la SVD dei due spettrogrammi, ricavando dalle colonne della matrice U gli autovettori richiesti, con l'accortezza di non sceglierli tutti, in quanto così facendo avremmo potuto ottenere ricostruzioni perfette di ogni segnale per entrambi i sottospazi, impedendo una classificazione efficace. Al contrario, poichè ad ogni autovettore è associato un autovalore che rappresenta la sua energia, abbiamo selezionato solo i primi autovettori in ordine di energia, affinchè la somma dei loro autovalori fosse pari al 50% del totale.

In [71]:
energy = 50
base_music = get_base_from_spectrogram(spectrogram_music,energy)
base_speech = get_base_from_spectrogram(spectrogram_speech,energy)

Una volta ottenute le due basi, abbiamo caricato i 10 file incogniti, e per ognuno di essi abbiamo calcolato lo spettrogramma (con le impostazioni precedenti). Successivamente abbiamo calcolato per ognuno di essi la proiezione dello spettrogramma su entrambi i sottospazi attraverso i prodotti scalari tra le colonne dello spettrogramma e gli autovettori della classe. A questo punto abbiamo calcolato le due matrici errore come la differenza tra la proiezione del file incognito su un sottospazio e lo spettrogramma originale, e ne abbiamo calcolato le norme quadratiche (o di Frobenius); la categoria la cui matrice errore aveva norma minore era quella che meglio rappresentava il segnale incognito.

In [72]:
unknowns = load_audio(path_unknowns)
for rate,data,filename in unknowns:
    current_spectrogram = get_spectrogram(data , rate)
    print('Item: ',filename)
    
    err_on_speech = get_error_base(current_spectrogram,base_speech)
    err_on_music = get_error_base(current_spectrogram,base_music)
    print('\tMetodo1:: ','MUSICA' if err_on_music<err_on_speech else'PARLATO')

Item:  004_BobDylan-OxfordTown-10s-B.wav
	Metodo1::  PARLATO
Item:  007si1079.wav
	Metodo1::  PARLATO
Item:  007si1271.wav
	Metodo1::  PARLATO
Item:  010_CiboMatto-Moonchild-10s-A.wav
	Metodo1::  MUSICA
Item:  014si1291.wav
	Metodo1::  PARLATO
Item:  016si1621.wav
	Metodo1::  PARLATO
Item:  019_Nirvana-Downer-10s-B.wav
	Metodo1::  MUSICA
Item:  024_JimiHendrix-WaitUntilTomorrow-10s-A.wav
	Metodo1::  MUSICA
Item:  025_Lana-SadGirl.wav
	Metodo1::  MUSICA
Item:  026_agnel.wav
	Metodo1::  PARLATO


<h3>Secondo metodo</h3>

Per il secondo metodo si procede allo stesso modo del precedente fino ad ottenere le due basi di autovettori per le classi di musica e di parlato.
A questo punto, invece che procedere con la proiezione dei suoni incogniti, si ottiene una 'feature' per ognuna delle due classi: una 'feature' è un elemento che rappresenta in modo sintetico l'intera classe. Per ottenere questa 'feature' abbiamo calcolato il vettore dei coefficienti della proiezione di ognuno dei vettori dello spettrogramma di una classe musica sulla base di autovettori della stessa classe, e poi abbiamo calcolato il vettore medio tra tutti i vettori dei coefficienti. Questi due vettori medi, uno per classe, rappresentano le nostre 'feature' per le due categorie.

In [73]:
feature_mean_music = get_feature_mean_from_list(music,base_music)
feature_mean_speech = get_feature_mean_from_list(speech,base_speech)

Passando quindi ai file incogniti, abbiamo calcolato lo spettrogramma di ognuno, e abbiamo proceduto come con lo spettrogramma delle due classi, calcolandone il vettore medio dei coefficienti. Quindi, per decidere quale categoria è quella corretta, abbiamo calcolato i due vettori errore come differenza tra il vettore medio del file incognito e la 'feature' della classe, e quello che possedeva norma quadratica minore era quello della classe giusta.

In [74]:
for rate,data,filename in unknowns:
    current_spectrogram = get_spectrogram(data , rate)
    print('Item: ',filename)
    
    err_on_speech = np.linalg.norm(feature_mean_speech-get_feature_mean(current_spectrogram,base_speech))
    err_on_music = np.linalg.norm(feature_mean_music-get_feature_mean(current_spectrogram,base_music))
    print('\tMetodo2:: ','MUSICA' if err_on_music<err_on_speech else'PARLATO')

Item:  004_BobDylan-OxfordTown-10s-B.wav
	Metodo2::  MUSICA
Item:  007si1079.wav
	Metodo2::  PARLATO
Item:  007si1271.wav
	Metodo2::  PARLATO
Item:  010_CiboMatto-Moonchild-10s-A.wav
	Metodo2::  MUSICA
Item:  014si1291.wav
	Metodo2::  PARLATO
Item:  016si1621.wav
	Metodo2::  PARLATO
Item:  019_Nirvana-Downer-10s-B.wav
	Metodo2::  MUSICA
Item:  024_JimiHendrix-WaitUntilTomorrow-10s-A.wav
	Metodo2::  MUSICA
Item:  025_Lana-SadGirl.wav
	Metodo2::  MUSICA
Item:  026_agnel.wav
	Metodo2::  PARLATO


<h3>Terzo metodo</h3>

Per il terzo metodo si procede sulla falsa riga del secondo, ma al contrario di quest'ultimo si utilizza una sola classe che include sia i file di musica che quelli di parlato. In pratica si concatenano gli spettrogrammi delle due categorie calcolati in precedenza, in modo da formare un unico spettrogramma, da cui si ricava la base di autovettori tramite la SVD nello stesso modo con cui la si era ottenuta per le singole classi.

In [75]:
spectrogram_total = scipy.hstack((spectrogram_music, spectrogram_speech))

energy = 50
base_total = get_base_from_spectrogram(spectrogram_total,energy)

A questo punto, si ottengono le 'feature' delle due categorie calcolando i vettori dei coefficienti della proiezione di ogni file di una delle due categorie sulla classe unica, e calcolandone il vettore medio (come fatto anche nel secondo metodo):

In [76]:
feature_mean_music_on_total = get_feature_mean_from_list(music,base_total)
feature_mean_speech_on_total = get_feature_mean_from_list(speech,base_total)

Si procede quindi con i file incogniti, calcolandone come prima il vettore medio dei coefficienti della proiezione sulla classe unica. Per decidere quale categoria è quella corretta, si calcolano i due vettori errore come differenza tra il vettore medio del file incognito e la 'feature' della categoria, prediligendo quello che possiede norma quadratica minore.

In [77]:
for rate,data,filename in unknowns:
    current_spectrogram = get_spectrogram(data , rate)
    print('Item: ',filename)
    
    err_on_speech = np.linalg.norm(feature_mean_speech_on_total-get_feature_mean(current_spectrogram,base_total))
    err_on_music = np.linalg.norm(feature_mean_music_on_total-get_feature_mean(current_spectrogram,base_total))
    print('\tMetodo3:: ','MUSICA' if err_on_music<err_on_speech else'PARLATO')

Item:  004_BobDylan-OxfordTown-10s-B.wav
	Metodo3::  MUSICA
Item:  007si1079.wav
	Metodo3::  PARLATO
Item:  007si1271.wav
	Metodo3::  PARLATO
Item:  010_CiboMatto-Moonchild-10s-A.wav
	Metodo3::  MUSICA
Item:  014si1291.wav
	Metodo3::  PARLATO
Item:  016si1621.wav
	Metodo3::  PARLATO
Item:  019_Nirvana-Downer-10s-B.wav
	Metodo3::  MUSICA
Item:  024_JimiHendrix-WaitUntilTomorrow-10s-A.wav
	Metodo3::  MUSICA
Item:  025_Lana-SadGirl.wav
	Metodo3::  MUSICA
Item:  026_agnel.wav
	Metodo3::  PARLATO


<h3>Analisi dei risultati</h3>

Per quanto riguarda il primo metodo, utilizzando il 50% dell'energia nel calcolo della base di autovettori, si ottiene una classificazione corretta per tutti i file audio eccetto il primo. La causa di questo fatto è dovuta probabilmente alla natura acustica del brano, in cui il cantante è accompagnato dalla sola chitarra. Questa particolarità, che non si riscontra negli altri brani musicali della collezione, rende l'audio più simile al parlato e trae in inganno il sistema di classificazione.
Provando a diminuire il livello di energia, i risultati restano uguali. Ad esempio, con una percentuale del 30% si ottiene:

In [80]:
energy = 30
base_music = get_base_from_spectrogram(spectrogram_music,energy)
base_speech = get_base_from_spectrogram(spectrogram_speech,energy)

unknowns = load_audio(path_unknowns)
for rate,data,filename in unknowns:
    current_spectrogram = get_spectrogram(data , rate)
    print('Item: ',filename)
    
    err_on_speech = get_error_base(current_spectrogram,base_speech)
    err_on_music = get_error_base(current_spectrogram,base_music)
    print('\tMetodo1:: ','MUSICA' if err_on_music<err_on_speech else'PARLATO')

Item:  004_BobDylan-OxfordTown-10s-B.wav
	Metodo1::  PARLATO
Item:  007si1079.wav
	Metodo1::  PARLATO
Item:  007si1271.wav
	Metodo1::  PARLATO
Item:  010_CiboMatto-Moonchild-10s-A.wav
	Metodo1::  PARLATO
Item:  014si1291.wav
	Metodo1::  PARLATO
Item:  016si1621.wav
	Metodo1::  PARLATO
Item:  019_Nirvana-Downer-10s-B.wav
	Metodo1::  PARLATO
Item:  024_JimiHendrix-WaitUntilTomorrow-10s-A.wav
	Metodo1::  PARLATO
Item:  025_Lana-SadGirl.wav
	Metodo1::  PARLATO
Item:  026_agnel.wav
	Metodo1::  PARLATO


Al contrario, aumentando il livello di energia i risultati sono via via meno precisi, tendendo a riconoscere come musica anche gli audio di parlato. Ad esempio, con l'80% di energia risulta:

In [82]:
energy = 80
base_music = get_base_from_spectrogram(spectrogram_music,energy)
base_speech = get_base_from_spectrogram(spectrogram_speech,energy)

unknowns = load_audio(path_unknowns)
for rate,data,filename in unknowns:
    current_spectrogram = get_spectrogram(data , rate)
    print('Item: ',filename)
    
    err_on_speech = get_error_base(current_spectrogram,base_speech)
    err_on_music = get_error_base(current_spectrogram,base_music)
    print('\tMetodo1:: ','MUSICA' if err_on_music<err_on_speech else'PARLATO')

Item:  004_BobDylan-OxfordTown-10s-B.wav
	Metodo1::  MUSICA
Item:  007si1079.wav
	Metodo1::  MUSICA
Item:  007si1271.wav
	Metodo1::  MUSICA
Item:  010_CiboMatto-Moonchild-10s-A.wav
	Metodo1::  MUSICA
Item:  014si1291.wav
	Metodo1::  MUSICA
Item:  016si1621.wav
	Metodo1::  MUSICA
Item:  019_Nirvana-Downer-10s-B.wav
	Metodo1::  MUSICA
Item:  024_JimiHendrix-WaitUntilTomorrow-10s-A.wav
	Metodo1::  MUSICA
Item:  025_Lana-SadGirl.wav
	Metodo1::  MUSICA
Item:  026_agnel.wav
	Metodo1::  MUSICA


Il problema di prendere un numero così elevato di autovettori è quello che entrambi gli spazi vettoriali tendono a ricostruire perfettamente tutti i segnali (cosa che avverrebbe perfettamente nel caso di una base completa), inficiando i risultati del programma.

Il secondo metodo risulta essere molto più solido rispetto al valore di energia scelto, e riconosce in maniera corretta tutti gli audio anche aumentando il livello di energia. Ad esempio, con il 100% dell'energia si ottiene:

In [85]:
energy = 100
base_music = get_base_from_spectrogram(spectrogram_music,energy)
base_speech = get_base_from_spectrogram(spectrogram_speech,energy)

feature_mean_music = get_feature_mean_from_list(music,base_music)
feature_mean_speech = get_feature_mean_from_list(speech,base_speech)

for rate,data,filename in unknowns:
    current_spectrogram = get_spectrogram(data , rate)
    print('Item: ',filename)
    
    err_on_speech = np.linalg.norm(feature_mean_speech-get_feature_mean(current_spectrogram,base_speech))
    err_on_music = np.linalg.norm(feature_mean_music-get_feature_mean(current_spectrogram,base_music))
    print('\tMetodo2:: ','MUSICA' if err_on_music<err_on_speech else'PARLATO')

Item:  004_BobDylan-OxfordTown-10s-B.wav
	Metodo2::  MUSICA
Item:  007si1079.wav
	Metodo2::  PARLATO
Item:  007si1271.wav
	Metodo2::  PARLATO
Item:  010_CiboMatto-Moonchild-10s-A.wav
	Metodo2::  MUSICA
Item:  014si1291.wav
	Metodo2::  PARLATO
Item:  016si1621.wav
	Metodo2::  PARLATO
Item:  019_Nirvana-Downer-10s-B.wav
	Metodo2::  MUSICA
Item:  024_JimiHendrix-WaitUntilTomorrow-10s-A.wav
	Metodo2::  MUSICA
Item:  025_Lana-SadGirl.wav
	Metodo2::  MUSICA
Item:  026_agnel.wav
	Metodo2::  PARLATO


Lo stesso discorso vale per il terzo metodo, in linea teorica il più robusto. Infatti, con il 100% dell'energia le classificazioni risultano ancora tutte corrette:

In [86]:
energy = 100
base_total = get_base_from_spectrogram(spectrogram_total,energy)

feature_mean_music_on_total = get_feature_mean_from_list(music,base_total)
feature_mean_speech_on_total = get_feature_mean_from_list(speech,base_total)

for rate,data,filename in unknowns:
    current_spectrogram = get_spectrogram(data , rate)
    print('Item: ',filename)
    
    err_on_speech = np.linalg.norm(feature_mean_speech_on_total-get_feature_mean(current_spectrogram,base_total))
    err_on_music = np.linalg.norm(feature_mean_music_on_total-get_feature_mean(current_spectrogram,base_total))
    print('\tMetodo3:: ','MUSICA' if err_on_music<err_on_speech else'PARLATO')

Item:  004_BobDylan-OxfordTown-10s-B.wav
	Metodo3::  MUSICA
Item:  007si1079.wav
	Metodo3::  PARLATO
Item:  007si1271.wav
	Metodo3::  PARLATO
Item:  010_CiboMatto-Moonchild-10s-A.wav
	Metodo3::  MUSICA
Item:  014si1291.wav
	Metodo3::  PARLATO
Item:  016si1621.wav
	Metodo3::  PARLATO
Item:  019_Nirvana-Downer-10s-B.wav
	Metodo3::  MUSICA
Item:  024_JimiHendrix-WaitUntilTomorrow-10s-A.wav
	Metodo3::  MUSICA
Item:  025_Lana-SadGirl.wav
	Metodo3::  MUSICA
Item:  026_agnel.wav
	Metodo3::  PARLATO


<h3>Conclusioni</h3>

In conclusione, i risultati confermano la buona riuscita dell'esperienza, che con due metodi su tre permette un perfetto riconoscimento degli audio, indipendentemente dalla percentuale di energia utilizzata nel formare la base delle due classi musica e parlato. 
Al contrario degli ultimi due, il primo metodo risulta invece meno preciso, dando la migliore classificazione con il 50% di energia: con questa impostazione il risultato è corretto per 11 file su 12, con l'unico errore per la canzone di Bob Dylan, acustica e quindi con caratteristiche simili al parlato.
La migliore riuscita degli ultimi due metodi è da imputare al fatto che le feature risultano molto diverse tra loro, ed è quindi più semplice per il programma distinguere a quale delle due classi appartiene un audio.